In [268]:
import pandas as pd
import random
import numpy as np


ff2020 = pd.read_csv("fantasy/fantasy_data_2020.csv")
ff2021 = pd.read_csv("fantasy/fantasy_data_2021.csv")

In [277]:
raw_scores = ff2021[['team_name', 'week', 'points']].drop_duplicates()
raw_scores.week.max()

2

In [278]:
def split(df, col):
    return [x for _, x in df.groupby(col)]

def create_team_dict(raw_scores):
    '''Create dictionary of team names and their scores'''
    team_scores = split(raw_scores, 'team_name')
    team_dict = {}
    for i in team_scores:
        team_name = i.team_name.drop_duplicates().values[0]
        team_dict.update({team_name:i.points.values})
    return team_dict

def simulate_season(team_name, team_dict):
    opponents = [i for i in list(team_dict.keys()) if i != team_name]
    # Randomize order of opponents

    # Get maximum number of games so far
    games_count = len(team_dict[team_name])

    all_wins = []
    for i in range(10000):
        new_order = random.sample(opponents, games_count) 
        wins = 0
        for j, opp in enumerate(new_order):
            my_points = team_dict[team_name][j]
            opp_points = team_dict[opp][j]
            if my_points > opp_points:
                wins += 1
        all_wins.append(wins)
    return all_wins

def build_probability_distribution(raw_scores):
    team_dict = create_team_dict(raw_scores)
    all_team_names = sorted(list(set(raw_scores.team_name.values)))
    simulated_wins = []
    for name in all_team_names:
        total_wins = pd.Series(simulate_season(name, team_dict))
        total_wins = total_wins.rename(name)
        simulated_wins.append(total_wins.value_counts() / total_wins.value_counts().sum())

    return pd.concat(simulated_wins, axis=1).fillna(0)

In [ ]:
liklihood_table = build_probability_distribution(raw_scores).sort_index(ascending=False).cumsum(axis=0).sort_index(ascending=True)
format_dict = {col:'{:,.1%}'.format for col in liklihood_table.columns }

In [297]:
liklihood_table.style.format(format_dict)

,12th in Name Power Rankings,49ers QB of the Future,ACL Count: 1,Born Again Christian,Derrickle McNichols,Edward Jordyhands,Finding Deebo,Jerry Jones' Blood Boy,Joe Rogan on Horse Dewormer,Murray Up Offense,Qaptain Qirk And the Qrew,The Salty Spitoon
0,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%,100.0%
1,55.0%,27.2%,32.5%,79.7%,93.0%,94.4%,100.0%,91.3%,74.2%,100.0%,74.3%,92.0%
2,8.7%,0.0%,3.7%,29.0%,51.8%,59.9%,36.7%,0.0%,6.9%,54.4%,24.6%,7.7%


In [161]:
win_probs = []
for x,i  in enumerate(dfs):
    i['win_probability'] = i['points'].rank() / i['points'].count()
    i['week'] = x
    win_probs.append(i[['team_name', 'win_probability', 'week']])


In [162]:
all_probs = pd.concat(win_probs)

In [163]:
team_split = split(all_probs, 'team_name')

In [164]:
def simulate_season(my_team_probs):
    total_wins = []
    for i in my_team_probs:
        total_wins.append(np.random.binomial(1,i))
    return total_wins


In [179]:
team_names = sorted(list(all_probs.team_name.drop_duplicates()))
team_split

[      team_name  win_probability  week
 6    49ers 14-6         0.166667     0
 45   49ers 14-6         0.250000     1
 78   49ers 14-6         0.333333     2
 117  49ers 14-6         0.833333     3
 150  49ers 14-6         0.500000     4
 189  49ers 14-6         0.083333     5
 222  49ers 14-6         0.250000     6
 261  49ers 14-6         0.083333     7
 294  49ers 14-6         0.083333     8
 327  49ers 14-6         0.500000     9
 366  49ers 14-6         0.500000    10
 405  49ers 14-6         0.416667    11
 441  49ers 14-6         0.083333    12,
             team_name  win_probability  week
 3    American Russell         0.250000     0
 36   American Russell         0.583333     1
 75   American Russell         0.916667     2
 108  American Russell         0.083333     3
 144  American Russell         0.416667     4
 183  American Russell         0.666667     5
 216  American Russell         0.750000     6
 255  American Russell         0.833333     7
 288  American Russell   

In [180]:
def simulate_wins_per_team(split_probs):
    '''Simulate 10000 seasons for the team, win_prob, week dataframe'''
    my_team_probs = split_probs.win_probability.to_list()
    simulated_season_wins = []
    for i in range(0,10000):
        simulated_season_wins.append(sum(simulate_season(my_team_probs)))
    wins_count = pd.Series(simulated_season_wins).value_counts().sort_index()
    probs = wins_count / wins_count.sum()
    #cusum_wins = probs.sort_index(ascending=False).cumsum().rename('probability')
    return probs

In [181]:
all_team_sims = []
for team in team_split:
    all_team_sims.append(simulate_wins_per_team(team))
full_season_all_probs = pd.concat(all_team_sims, axis=1).fillna(0)
full_season_all_probs.columns = team_names
full_season_all_probs.sort_index(ascending=False).cumsum().sort_index(ascending=True)


,49ers 14-6,American Russell,Clapped Cheeks,Double D Kupps,Edward Jordyhands,Finding Deebo,It Ertz When I Pee,Smoke Herb Everyday,The Mahomies,The Salty Spitoon,Tua Legit Tua Quit,¯\_(ツ)_/¯ ¯\_(ツ)_/¯
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9974,1.0000,0.9998,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.9998,1.0000
2,0.9688,1.0000,0.9964,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.9977,0.9996
3,0.8635,0.9979,0.9700,1.0000,1.0000,0.9992,1.0000,1.0000,1.0000,1.0000,0.9769,0.9955
4,0.6459,0.9847,0.8735,1.0000,1.0000,0.9909,0.9934,0.9999,0.9975,0.9997,0.9106,0.9729
5,0.3746,0.9249,0.6721,0.9993,1.0000,0.9535,0.9446,0.9945,0.9802,0.9987,0.7544,0.8999
6,0.1640,0.7761,0.4040,0.9841,0.9996,0.8315,0.7918,0.9682,0.9135,0.9886,0.5178,0.7468
7,0.0532,0.5326,0.1789,0.9015,0.9945,0.6065,0.5423,0.8820,0.7485,0.9520,0.2775,0.5259
8,0.0133,0.2694,0.0557,0.6872,0.9680,0.3464,0.2773,0.6888,0.5036,0.8394,0.1068,0.2922
9,0.0019,0.0898,0.0115,0.4014,0.8868,0.1413,0.1009,0.4159,0.2532,0.6262,0.0301,0.1226


In [121]:
import altair as alt

In [131]:
alt.Chart(cusum_wins.reset_index()).mark_line().encode(x='index', y='probability', tooltip=[alt.Tooltip('index', title='Total H2H Wins'), 'probability']).interactive()

alt.Chart(...)